In [1]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
from mpl_toolkits.basemap import Basemap
from scipy.stats import norm
import imageio
import os

#### The longitudes and latitudes variables will be same for each dataset file. So we can extract the latitudes and longitudes once and then create meshgrid using them. Then meshgrid,lats,lons can be used multiple times.

In [2]:
# Code to extract the latitudes and longitudes from a file
dset = nc.Dataset("basic_dataset.nc")

longitudes = np.array(dset.variables["LONGITUDE"][:].data)
latitudes = np.array(dset.variables["LATITUDE"][:].data)

lons, lats = np.meshgrid(longitudes, latitudes)

## Codes for preparing the data of a single pair of zonal and meridonal file for creation of quiver plot

In [3]:
# Function to get the data and prepare it for creating plots
def prepareData(file_path_zonal, file_path_meridonal):
    
    # Creating the datasets from the files
    dset_zonal = nc.Dataset(file_path_zonal)
    dset_meridonal = nc.Dataset(file_path_meridonal)

    # Extracting the uSpeed and vSpeed as numpy arrays.
    uSpeed = np.array(dset_zonal.variables["U"][:].data)
    vSpeed = np.array(dset_meridonal.variables["V"][:].data)

    #Converting from 3D to 2D
    uSpeed = uSpeed[0] 
    vSpeed = vSpeed[0]

    # setting the missing speed values as nan
    vSpeed[vSpeed == -1e+34] = np.nan
    uSpeed[uSpeed == -1e+34] = np.nan

    return (uSpeed, vSpeed)


### Code For creating a single quiver plots
- The stride, length and whether vector sizes are same or not all these values are taken as parameters of the functions which can be tweaked.

In [4]:
# Function for creating quiver plots
def wind_speed_quiver(file_path_zonal, file_path_meridonal, stride, sameSize, length, output_folder, imageName, lons=lons, lats=lats):

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # get the u anv v speeds
    u,v = prepareData(file_path_zonal, file_path_meridonal)

    m = Basemap(projection='cyl', resolution='c', llcrnrlat=-90, urcrnrlat=90, llcrnrlon=0, urcrnrlon=360)
    
    # Create a figure and axis
    fig = plt.figure(figsize=(120, 80))
    ax1 = fig.add_subplot(1, 1, 1)

    # Draw the coastlines and countries
    m.drawcoastlines()
    m.drawcountries()

    # Fill continents with grey color
    m.fillcontinents(color='lightgrey')

    # Convert longitude and latitude to map coordinates
    lons, lats = m(lons, lats)

    # Create the quiver plot
    u = u[::stride, ::stride].squeeze()
    v = v[::stride, ::stride].squeeze()

    if sameSize:
        # Calculate the magnitude of the wind vector
        magnitude = np.sqrt(u**2 + v**2)

        # Normalize u and v
        u = u / magnitude
        v = v / magnitude

        # Set a fixed length for all arrows
        u = u * length
        v = v * length

    magnitude = np.sqrt(u**2 + v**2)

    quiver = m.quiver(lons[::stride, ::stride], lats[::stride, ::stride], u, v,magnitude, width= 0.0009,scale = 500, color="blue")

    # Add labels and title
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.set_title('Wind Speed quiver plot with World Map')


    # Save the plot as an image file
    output_file = os.path.join(output_folder, f'quiver_plot_{imageName}.png')
    plt.savefig(output_file)
    plt.close()


### Code for creating all the quiver images using the data files of various dates. Then using this data to create gif

In [5]:
def createAllQuiverImages(folder_path_zonal, folder_path_meridonal, stride, sameSize, length, imagesFolderName):
    
    # Get the file names with and without the extension of .nc
    file_names = sorted([file for file in os.listdir(folder_path_zonal) if file.endswith('.nc')])
    file_names_without_extension = [file[:-3] for file in file_names]



    # Create and store the images one by one by looping over all the file
    for i, file_name in enumerate(file_names):
        #construct the full path names
        zonal_file_path = os.path.join(folder_path_zonal, file_name)
        meridonal_file_path = os.path.join(folder_path_meridonal, file_name)

        wind_speed_quiver(zonal_file_path, meridonal_file_path, stride= stride, sameSize = sameSize, length=length, output_folder = imagesFolderName, imageName=file_names_without_extension[i])
    
createAllQuiverImages("./november2013_data/zonal/", "./november2013_data/meridonal/", 7, False, 5, "quiver_images")


In [6]:
# code for creating the gifs
def create_gif(input_folder, output_file):
    images = []

    # Get the list of image files in the input folder
    image_files = sorted([file for file in os.listdir(input_folder) if file.endswith('.png')])

    for image_file in image_files:
        image_path = os.path.join(input_folder, image_file)
        images.append(imageio.imread(image_path))

    # Save the images as a GIF
    imageio.mimsave(output_file, images, duration=2.7)

# Specify the input folder and output GIF file
input_folder = 'quiver_images'
output_gif = 'quiver_animation.gif'

# Create the GIF
create_gif(input_folder, output_gif)

print(f'GIF created successfully: {output_gif}')


<ipython-input-6-8b1d8fd07f66>:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(image_path))


GIF created successfully: quiver_animation.gif
